In [9]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup

In [2]:
CWD = os.getcwd()

In [14]:
def get_current_currency_rate(currency_from: str, currency_to: str) -> float:
    """
    This method is used to scrap real time currency from x-rates.com. This method return us rate of given comparison
    @param currency_from: <str> currency from
    @param currency_to: <str> currency to
    return rate: <float> rate of target currency
    """
    currency_from = currency_from.upper()
    currency_to = currency_to.upper()
    request_url: str = f"https://www.x-rates.com/calculator/?from={currency_from}&to={currency_to}&amount=1"
    with requests.Session() as session:
        response = session.get(url=request_url)
        soup_obj = BeautifulSoup(response.text, "html.parser")

    part1 = soup_obj.find(class_="ccOutputTrail").previous_sibling
    part2 = soup_obj.find(class_="ccOutputTrail").get_text(strip=True)
    rate = "{}{}".format(part1, part2)
    return float(rate)

In [15]:
get_current_currency_rate('USD', 'TRY')

23.654252

In [92]:
def get_currency_by_day(currency_from: str, date: str):
    c_from = currency_from.upper()
    date = date.upper()
    request_url: str = f"https://www.x-rates.com/historical/?from={c_from}&amount=1&date={date}"
    with requests.Session() as session:
        response = session.get(url=request_url)
        soup_obj = BeautifulSoup(response.text, "html.parser")
    # parse html
    table = soup_obj.find('table', {
        'class': 'ratesTable'
    }).find('tbody').find_all('tr')
    for item in table:
        data = item.find_all('td')
        currency_name = data[0].text
        currency_amount = data[1].find('a').text
        inv_currency_amount = data[2].find('a').text
        print(currency_name, currency_amount, inv_currency_amount)

In [93]:
get_currency_by_day(currency_from='USD', date='2020-06-14')

Euro 0.888573 1.125400
British Pound 0.797237 1.254332
Indian Rupee 75.947557 0.013167
Australian Dollar 1.457916 0.685911
Canadian Dollar 1.358219 0.736258
Singapore Dollar 1.392472 0.718147
Swiss Franc 0.952163 1.050240
Malaysian Ringgit 4.270633 0.234157
Japanese Yen 107.384826 0.009312
Chinese Yuan Renminbi 7.081075 0.141221
